In [1]:
import pathlib

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
ROOT = pathlib.Path.home() / ".data/mnist"

# https://github.com/pytorch/examples/blob/master/mnist/main.py
# https://discuss.pytorch.org/t/normalization-in-the-mnist-example/457/2
tsfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_ds = datasets.MNIST(str(ROOT), download=True, train=True, transform=tsfms)
test_ds = datasets.MNIST(str(ROOT), download=True, train=False, transform=tsfms)

# Hyperparameters

In [ ]:
batch_size = 300
num_epochs = 100
learning_rate = 0.001

In [5]:
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
test_dl = DataLoader(test_ds, batch_size=batch_size)
criterion = nn.CrossEntropyLoss()

In [6]:
from torchvision.models.resnet import BasicBlock, conv3x3

In [7]:
def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [8]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_input_channels=3, num_classes=1000, zero_init_residual=False):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(num_input_channels, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [9]:
model = ResNet(BasicBlock, [2, 2, 2, 2], num_input_channels=1)

In [10]:
model = model.to(device)

In [11]:
optimizer = torch.optim.Adam(model.parameters())

In [12]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

logger = logging.getLogger("training")

In [13]:
model.train()
for epoch in range(num_epochs):
    logger.info(f"Starting epoch {epoch}")
    for i, (x, y) in enumerate(train_dl):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        unnormalized_logits = model(x)
        loss = criterion(unnormalized_logits, y)
        loss.backward()
        optimizer.step()
    logger.info(f"Finished epoch {epoch}")

model.eval()
correct = 0
total = 0
for x, y in test_dl:
    x = x.to(device)
    y = y.to(device)
    unnormalized_logits = model(x)
    max_value, predicted = torch.max(unnormalized_logits, dim=1)
    total += y.size(0)
    correct += (predicted == y).sum()

correct = int(correct.data)
print(f"{correct} / {total} {correct / total}")

2019-02-09 20:06:05,384 - training - INFO - Starting epoch 0
2019-02-09 20:11:14,432 - training - INFO - Finished epoch 0
2019-02-09 20:11:14,434 - training - INFO - Starting epoch 1
2019-02-09 20:16:09,307 - training - INFO - Finished epoch 1
2019-02-09 20:16:09,309 - training - INFO - Starting epoch 2


Traceback (most recent call last):
  File "/Users/allenhuang/miniconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/Users/allenhuang/miniconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/allenhuang/miniconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/allenhuang/miniconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/Users/allenhuang/miniconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/Users/allenhuang/miniconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/allenhuang/miniconda3/lib/python3.7/multiprocessing/connectio

KeyboardInterrupt: 